In [15]:
import os
import sqlite3
import pandas as pd
from dotenv import load_dotenv

conn = sqlite3.connect("etl/soccer_analysis.db")

In [41]:
query = '''
SELECT *
FROM "2024_2025" AS st17
LEFT JOIN "possession_2024_2025" AS ps17
  ON st17.Player = ps17.Player AND st17.Squad = ps17.Squad AND st17.Nation = ps17.Nation
LEFT JOIN "gca_2024_2025" AS gca17
  ON st17.Player = gca17.Player AND st17.Squad = gca17.Squad AND st17.Nation = gca17.Nation
'''
df = pd.read_sql_query(query, conn)


In [42]:
df = df.loc[:,~df.columns.duplicated()]

In [43]:
df.columns

Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', 'MP',
       'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt',
       'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR',
       'Gls_90', 'Ast_90', 'G+A_90', 'G-PK_90', 'G+A-PK', 'xG_90', 'xAG_90',
       'xG+xAG', 'npxG_90', 'npxG+xAG_90', 'Matches', 'Touches', 'Def Pen',
       'Def 3rd', 'Mid 3rd', 'Att 3rd', 'Att Pen', 'Live', 'Att', 'Succ',
       'Succ%', 'Tkld', 'Tkld%', 'Carries', 'TotDist', 'PrgDist', '1/3', 'CPA',
       'Mis', 'Dis', 'Rec', 'SCA', 'SCA90', 'PassLive', 'PassDead', 'TO', 'Sh',
       'Fld', 'Def', 'GCA', 'GCA90', 'PassLive_90', 'PassDead_90', 'TO_90',
       'Sh_90', 'Fld_90', 'Def_90'],
      dtype='object')

In [44]:
df = df[
    (
        (df['PrgC'] >= df['PrgC'].max() * 0.65) | (df['xAG'] >= df['xAG'].max() * 0.65)
    )
]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Min-Max normalization
df['xAG_norm'] = (df['xAG'] - df['xAG'].min()) / (df['xAG'].max() - df['xAG'].min())
df['PrgC_norm'] = (df['PrgC'] - df['PrgC'].min()) / (df['PrgC'].max() - df['PrgC'].min())

plt.figure(figsize=(12, 8))
ax = sns.scatterplot(data=df, x='xAG_norm', y='PrgC_norm')

for _, row in df.iterrows():
    ax.text(x=row['xAG_norm'], y=row['PrgC_norm'] - 0.03, s=row['Player'], fontsize=8, ha='center', va='center')

ax.set_xlabel('xAG (normalized)')
ax.set_ylabel('PrgC (normalized)')
plt.title('Normalized Scatterplot of xAG vs PrgC')
plt.show()

ValueError: Could not interpret value `Succ_norm` for `y`. An entry with this name does not appear in `data`.

<Figure size 1200x800 with 0 Axes>